In [ ]:
%cd ..
import os
import time

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects
from optimized_ingestion.utils.preprocess import preprocess
database.connection
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
NUSCENES_RAW_DATA = "NUSCENES_RAW_DATA"
if NUSCENES_RAW_DATA in os.environ:
    RAW_DATA_DIR = os.environ[NUSCENES_RAW_DATA]
else:
    RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini"

In [ ]:
import cv2
from os.path import exists
def construct_video(frames, size=(1600, 900), base=False, vid_name=None, vid_prefix=False):
    unique_frames = []
    for f in frames:
        if f not in unique_frames:
            unique_frames.append(f)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = "./outputs/fig13_"+vid_name if vid_name else f"./outputs/{'fig13_base' if base else 'fig13_optimized'}.mp4"
    display_video = cv2.VideoWriter(vid_name,fourcc, 1, size)
    for frame in unique_frames:
        if vid_prefix:
            img_path = os.path.join(RAW_DATA_DIR, 'sweeps/CAM_FRONT', frame)
            if not exists(img_path):
                img_path = os.path.join(RAW_DATA_DIR, 'samples/CAM_FRONT', frame)
        else:
            img_path = os.path.join(RAW_DATA_DIR, frame)
        img = cv2.imread(img_path)
        display_video.write(img)

    display_video.release()

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (obj1.id != obj2.id) &
    (F.like(obj1.type, 'car') | F.like(obj1.type, 'truck')) &
    (F.like(obj2.type, 'car') | F.like(obj2.type, 'truck')) &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 40, 135) &
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
    (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)


In [ ]:
preprocess(world, DATA_DIR, ['scene-0655-CAM_FRONT'], base=False)

In [ ]:
import numpy as np
investigation =[[0, 530, 480, 692, 584, 'intersection'], [1, 0, 452, 210, 647, 'intersection'], [2, 1329, 475, 1599, 612, 'intersection'], [3, 196, 430, 467, 598, 'intersection'], [4, 1105, 469, 1277, 547, 'intersection'], [5, 458, 476, 531, 550, 'intersection'], [6, 724, 480, 763, 510, 'intersection'], [7, 511, 470, 591, 532, 'intersection'], [8, 1292, 475, 1487, 579, 'intersection'], [9, 773, 473, 811, 498, 'intersection'], [10, 1055, 478, 1095, 507, 'intersection'], [11, 849, 476, 875, 500, 'intersection']]
target = [[0, 530, 480, 692, 584, 'intersection', 
           np.array([list(zip([363.0340999493258, 1337.7113240288093, 1692.9239863364926,
                               1790.0120384724512, 2032.4876728386516, 363.0340999493258],
                              [863.9872080248853, 869.7245561458105, 896.1907881034633,
                               933.9768149085215, 1035.6408551785412, 863.9872080248853]))]).reshape(-1,1,2)]]

skipping_info_0655=[[1, None, None], [2, None, None], [10, 'ego_exit_segment', None], [11, None, None], [12, None, None], [13, None, None], [14, None, None], [15, None, None], [16, None, None], [17, None, None], [20, 'meet_up', ((-133, 219), (135, 499))], [21, 'car_exit_segment', ((412, 429), (504, 501))], [22, None, None], [23, None, None], [24, None, None], [25, None, None], [26, None, None], [27, None, None], [34, 'car_exit_segment', ((521, 413), (595, 485))], [35, 'ego_exit_segment', None], [37, 'car_exit_segment', ((1084, 402), (1324, 516))], [46, 'car_exit_segment', ((1105, 392), (1367, 516))], [55, 'car_exit_segment', ((962, 444), (1132, 504))], [56, 'car_exit_segment', ((1019, 430), (1191, 508))], [57, 'car_exit_segment', ((454, 448), (538, 496))], [58, 'car_exit_segment', ((-25, 343), (293, 519))], [65, 'ego_exit_segment', None], [113, None, None], [114, None, None], [115, None, None], [116, None, None], [117, None, None], [118, None, None], [119, None, None], [120, None, None], [121, None, None], [122, None, None], [132, 'ego_exit_segment', None], [133, 'car_exit_segment', ((1199, 448), (1271, 486))], [137, 'car_exit_segment', ((651, 428), (727, 490))], [138, 'car_exit_segment', ((1173, 464), (1235, 502))], [140, 'car_exit_segment', ((644, 440), (724, 500))], [141, None, None], [142, None, None], [143, None, None], [153, 'ego_exit_segment', None], [167, None, None], [168, None, None], [169, None, None], [170, None, None], [171, None, None], [172, None, None], [173, None, None], [174, None, None], [175, None, None], [182, 'ego_exit_segment', None], [183, None, None], [184, None, None], [185, None, None], [186, None, None], [187, None, None], [188, None, None], [189, None, None], [195, 'ego_exit_segment', None], [209, 'car_exit_segment', ((700, 440), (756, 486))], [211, 'car_exit_segment', ((695, 440), (753, 484))], [213, 'car_exit_segment', ((681, 426), (745, 480))], [214, None, None], [215, None, None], [216, None, None], [217, 'car_exit_segment', ((17, 456), (141, 514))], [220, 'car_exit_segment', ((-11, 461), (115, 513))], [223, 'ego_exit_segment', None], [236, 'exit_view', ((563, 405), (685, 501))]]

def investigation_skipping(video_filename, skipping_info):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vid_name = f"./outputs/{video_filename}_skipping_investigation.mp4"
    display_video = cv2.VideoWriter(vid_name, fourcc, 1, (1600, 900))
    video_path = os.path.join(DATA_DIR, "videos", video_filename)
    vidcap = cv2.VideoCapture(video_path)
    frame_idx = 0
    current_frame_num, current_action, current_target_bbox = skipping_info[0]
    current_skipping_info_idx = 0
    while(vidcap.isOpened()):
        success, image = vidcap.read()
        if not success:
            break
        next_skipping_info_idx = current_skipping_info_idx + 1
        if next_skipping_info_idx < len(skipping_info):
            next_frame_num, next_action, next_target_bbox = skipping_info[next_skipping_info_idx]
        if frame_idx < next_frame_num:
            if current_action:
                image = cv2.putText(image, current_action, (50, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            if current_target_bbox:
                x, y = current_target_bbox[0]
                x_w, y_h = current_target_bbox[1]
                cv2.rectangle(image, (x+(x_w-x)//2, y+(y_h-y)//2), (x_w+(x_w-x)//2, y_h+(y_h-y)//2), (0,255,0), 4)
        if frame_idx == 55:
            for e in target:
                did, x, y, x_w, y_h, t, pts1 = e
                cv2.putText(image, f'obj_{did}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.putText(image, t, (x, y_h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(image, (x,y), (x_w, y_h), (0,255,0), 2)
                cv2.polylines(image, np.int32([pts1]), isClosed=True, color=(255,0,0), thickness = 2)
            cv2.imwrite('./outputs/0655_frame_55_skipping_investigation.jpg', image)
        display_video.write(image)
        frame_idx += 1
        if frame_idx == next_frame_num:
            current_skipping_info_idx = next_skipping_info_idx
            current_action = next_action
            current_target_bbox = next_target_bbox
    display_video.release()
investigation_skipping('boston-seaport-scene-0655-CAM_FRONT.mp4', skipping_info_0655)

In [ ]:
optimized_query_start = time.time()
id_time_camId_filename = world.get_id_time_camId_filename(2)
print(f'optimized query time: {time.time()-optimized_query_start}')

In [ ]:
optimized_results = [e[4] for e in id_time_camId_filename]
construct_video(optimized_results)

In [ ]:
preprocess(world, DATA_DIR, ['scene-0757-CAM_FRONT'])

In [ ]:
base_query_start = time.time()
id_time_camId_filename_base = world.get_id_time_camId_filename(2)
print(f'based query time: {time.time()-base_query_start}')

In [ ]:
id_time_camId_filename_base

In [ ]:
base_results = [e[4] for e in id_time_camId_filename_base]
construct_video(base_results, base=True)

In [ ]:
false_negative = [f for f in base_results if f not in optimized_results]
print(len(false_negative))
construct_video(false_negative, vid_name="false_negative.avi")

In [ ]:
false_positive = [f for f in optimized_results if f not in base_results]
print(len(false_positive))
construct_video(false_positive, vid_name="false_positive.avi")